<a href="https://colab.research.google.com/github/utpalmishra21/IBM-Applied-Data-Science-Capstone/blob/master/IBMDataScienceCapstonePart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###IMPORT Libraries

In [ ]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

###GET Request

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
print("Request Status: ", r)

Request Status:  <Response [200]>


###PARSING

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
data = soup.find_all('table')[0]
#data

###TEXT EXTRACTION

In [ ]:
PostalCode = []
Borough = []
Neighborhood = []

#data.find_all('tr')[0].find_all('th')[0].text.strip() #PostalCode
#data.find_all('tr')[0].find_all('th')[1].text.strip() #Borough
#data.find_all('tr')[0].find_all('th')[2].text.strip() #Neighborhood

for i in tqdm(range(1, len(data.find_all('tr')))):
  PostalCode.append(data.find_all('tr')[i].find_all('td')[0].text.strip())
  Borough.append(data.find_all('tr')[i].find_all('td')[1].text.strip())
  Neighborhood.append(data.find_all('tr')[i].find_all('td')[2].text.strip())

100%|██████████| 180/180 [00:00<00:00, 469.92it/s]


#DATAFRAME: Pre-Processing

In [ ]:
import pandas as pd

data = pd.DataFrame({'Postal Code':PostalCode,
                     'Borough':Borough,
                     'Neighborhood': Neighborhood})#, index = PostalCode)

In [ ]:
print(data.shape)
data.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [ ]:
data = data.sort_values(by = ['Postal Code'])
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn


###COURSERA Condition : Ignore cells with a borough that is Not assigned

In [ ]:
data = data[data['Borough'] != 'Not assigned']
print(data.shape)
data.head()

(103, 3)


,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


###CO-ORDINATES CSV

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Geospatial_Coordinates.csv'
coordinates = pd.read_csv(path)
coordinates = coordinates.sort_values(by = ['Postal Code'])
#coordinates.head()

In [ ]:
data = pd.merge(data, coordinates, on="Postal Code")
data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [ ]:
data.to_csv('Toronto.csv')
!cp Toronto.csv "drive/My Drive/Coursera/IBM"